<a href="https://colab.research.google.com/github/byiringiroscar/NLP_FELLOWSHIP/blob/main/oscar_assignment_igihe_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

====   here we are going to use api for wayback machine to help us to get all daily snapshoot

In [ ]:
snap = []
err = []
for month in range(1,11):
    for day in range(1,32):
        link = requests.get(url='http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}'.format(month, day))
        try:
            if link.json()['archived_snapshots']['closest']['available']:
              #we store each snapshot link into snap list
                snap.append(link.json()['archived_snapshots']['closest']['url'])
        except KeyError:
            err.append(link.json())
            

In [ ]:
len(snap)

301

===== downside we are going to remove the duplicate in all snappshoot captured

In [ ]:
snap_set  = set(snap)

In [ ]:
snap_r = list(snap_set)
len(snap_r)

195

==== here we are going to BeautifulSoup to get all content in all snapshoot

In [ ]:
#Fetch all the titles and their link from wayback archivr

links_art = []

for lin in snap_r:
  content = requests.get(lin).content #pass each snapshot link to the requests
  soup =  BeautifulSoup(content, "html.parser") #pass each snapshot content html parser
  articles = soup.find_all('span', class_="homenews-title") #look for all title in the snapshot
  links_art.append(articles)

In [ ]:
len(list(snap_r))

==== downside we are going to put all links in one side with their title in order to be zipped

In [ ]:
#Actual link
snapshot_link = []
title_ = []

#Loop all the 184 snapshots
for index, snapshot in enumerate(list(snap_r)):
  #In each title assign it's snapshop prefix link
  for title in links_art[index]:
    snapshot_link.append(snapshot)
    title_.append(str(title.find('a')['href']))

In [ ]:
len(snapshot_link)

=== here we are creating new dataframe with title and title_link

In [ ]:
df = pd.DataFrame(list(zip(snapshot_link, title_)), columns=['Prefix', 'Title'])

In [ ]:
df.head(5)

In [ ]:
df.describe()

=== here we are removing duplication from our list

In [ ]:
df.drop_duplicates(['Title'])

In [ ]:
link = df['Prefix'][0] + df['Title'][0]

content = requests.get(link).content

raw_content = BeautifulSoup(content, 'html.parser')

In [ ]:
body = raw_content.find_all('div', class_="fulltext margintop10") 

text = ''
par_ = BeautifulSoup(str(body)).find_all('p')

for line in par_:
  text = text + '\n' + line.get_text()



In [ ]:
def get_text(link):
  content = requests.get(link).content
  raw_content = BeautifulSoup(content, 'html.parser')

  body = raw_content.find_all('div', class_="fulltext margintop10") 

  text = ''
  par_ = BeautifulSoup(str(body)).find_all('p')

  for line in par_:
    text = text + '\n' + line.get_text()

In [ ]:
def get_article_text(link):
  content = requests.get(link).content
  body = BeautifulSoup(content, 'html.parser').find_all('div', class_='fulltext margintop10')

  text = ''
  try:
    par_ = BeautifulSoup(str(body)).find_all('p')


    for line in par_:
      text = text + '\n' + line.get_text()
  except AttributeError:
    pass
  return text

In [ ]:
articles = []

for index, article in enumerate(df['Title']):
  link =df['Prefix'][index]+article
  txt = get_article_text(link)
  articles.append(txt)

In [ ]:
=== here we are going to take all content of each link then save it in text

In [2]:
def get_all_text_file(index):
  link = df['Prefix'][index] + df['Title'][index]

  content = requests.get(link).content

  raw_content = BeautifulSoup(content, 'html.parser')
  body = raw_content.find_all('div', class_="fulltext margintop10") 

  text = ''
  par_ = BeautifulSoup(str(body)).find_all('p')

  for line in par_:
    text = text + '\n' + line.get_text()
  return text


In [ ]:
for index in range(len(df)):
  new_t = get_all_text_file(index)
  y = open('/content/drive/MyDrive/NLP Fellowship/huzalab_doc/week1/content_file.txt', 'w+',  encoding='utf-8')
  y.write(new_t+' \n')